In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
from itertools import permutations

import openai
from dotenv import load_dotenv

import art
from art.local import LocalBackend

load_dotenv()

backend = LocalBackend()
model = art.TrainableModel(
    name="009",
    project="yes-no-maybe",
    base_model="Qwen/Qwen2.5-7B-Instruct",
    # _internal_config=art.dev.InternalModelConfig(
    #     _decouple_vllm_and_unsloth=True,
    #     engine_args=art.dev.EngineArgs(gpu_memory_utilization=0.7),
    # ),
)
await model.register(backend)


async def rollout(client: openai.AsyncOpenAI, prompt: str) -> art.Trajectory:
    messages: art.Messages = [
        {
            "role": "user",
            "content": prompt,
        }
    ]
    chat_completion = await client.chat.completions.create(
        messages=messages, model=model.name, max_tokens=100, timeout=100
    )
    choice = chat_completion.choices[0]
    content = choice.message.content
    assert isinstance(content, str)
    if content == "yes":
        reward = 0.5
    elif content == "no":
        reward = 0.75
    elif content == "maybe":
        reward = 1.0
    else:
        reward = 0.0
    return art.Trajectory(messages_and_choices=[*messages, choice], reward=reward)


def with_quotes(w: str) -> str:
    return f"'{w}'"


prompts = [
    f"{prefix} with {', '.join([with_quotes(w) if use_quotes else w for w in words]) if len(words) == 3 else f'{words[0]}' + (f' or {words[1]}' if len(words) > 1 else '')}"
    for prefix in ["respond", "just respond"]
    for use_quotes in [True, False]
    for words in (
        list(p) for n in [3, 2] for p in permutations(["yes", "no", "maybe"], n)
    )
]

openai_client = model.openai_client()
for _ in range(await model.get_step(), 1_000):
    train_groups = await art.gather_trajectory_groups(
        (
            art.TrajectoryGroup(rollout(openai_client, prompt) for _ in range(32))
            for prompt in prompts
        )
    )
    await model.train(
        train_groups,
        config=art.TrainConfig(learning_rate=1e-4),
        # _config=art.dev.TrainConfig(
        #     precalculate_logprobs=True,
        # ),
    )

/root/art/.venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/root/art/.venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using t

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/art/src/art/__init__.py:10: UserWarning: WARNING: Unsloth should be imported before [transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore # noqa: F401


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 12-29 15:57:42 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
==((====))==  Unsloth 2025.12.5: Fast Qwen2 patching. Transformers: 4.57.3. vLLM: 0.12.0.
   \\   /|    NVIDIA H200. Num GPUs = 1. Max memory: 139.811 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-7b-instruct-unsloth-bnb-4bit with actual GPU utilization = 78.66%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 139.81 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 32768. Num Sequences = 256.
Unsloth: vLLM's KV Cache can use up to 104.11 GB. Also swap space = 6 GB.
Unsloth: Not an error, but `use_cudagr

(EngineCore_DP0 pid=1722586) /root/art/.venv/lib/python3.10/site-packages/pydantic/type_adapter.py:605: UserWarning: Pydantic serializer warnings:
(EngineCore_DP0 pid=1722586)   PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [field_name='mode', input_value=3, input_type=int])
(EngineCore_DP0 pid=1722586)   return self.serializer.to_python(


...

(EngineCore_DP0 pid=1722586) WARNING 12-29 15:58:09 [utils.py:250] Using default LoRA kernel configs


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 102/102 [00:11<00:00,  8.82it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 70/70 [00:06<00:00, 11.30it/s]


(EngineCore_DP0 pid=1722586) INFO 12-29 15:58:27 [gpu_model_runner.py:4466] Graph capturing finished in 18 secs, took 3.25 GiB
(EngineCore_DP0 pid=1722586) INFO 12-29 15:58:27 [core.py:254] init engine (profile, create kv cache, warmup model) took 29.59 seconds


/root/art/.venv/lib/python3.10/site-packages/unsloth_zoo/vllm_utils.py:851: RuntimeWarning: coroutine 'AsyncLLM.collective_rpc' was never awaited
  weights = llm.collective_rpc("get_weight_ipc_handles", args = tuple())[0]


RuntimeError: Unsloth: Cannot get internal vLLM states with error = 'coroutine' object is not subscriptable